# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [5]:
import sys
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("Flights")
 .getOrCreate())



In [6]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("C:/Users/alejandro.perez/Documents/Datasets/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [7]:
flightsDF.withColumn("Org_Dest", F.concat_ws("-","Origin","Dest"))\
              .withColumn("Delay", F.col("ArrDelay")+F.col("DepDelay"))\
              .groupBy(F.col("Org_Dest"))\
              .agg(F.mean(F.col("Delay")).alias("avgDelay"),F.count("Org_Dest").alias("NumeroVuelos")).sort(F.col("avgDelay").desc()).show()

+--------+------------------+------------+
|Org_Dest|          avgDelay|NumeroVuelos|
+--------+------------------+------------+
| RDM-MFR|            2790.0|           2|
| MDT-HPN|            1557.0|           1|
| ORD-GTF|             469.0|           1|
| ICT-DAY|             391.0|           1|
| ELM-ATL|             363.0|           2|
| DSM-PIA|             323.0|           1|
| ERI-ITH|             316.0|           1|
| CMH-HOU|             268.0|           1|
| YNG-PIE|             252.0|           1|
| PPG-HNL|220.54285714285714|          35|
| HNL-PPG|210.94285714285715|          35|
| AVP-SFB|             206.0|           1|
| HRL-DAL|             197.0|           1|
| PIE-YNG|             193.0|           1|
| CPR-LAS|             185.0|           1|
| MSP-PVD|             176.0|           1|
| ACY-MSY|171.54545454545453|          11|
| LAS-CPR|             163.0|           1|
| TTN-BNA|             153.7|          10|
| GRK-ATL|             131.7|          21|
+--------+-

In [8]:
AggFlights = cleanFlightsDF.groupBy(F.col("Origin"), F.col("Dest")).agg(
                                    F.round(F.mean(F.col("ArrDelay")),1).alias("Average_delay"),
                                    F.count("*").alias("Number_flights"))\
                                    .orderBy(F.col("Average_delay"), ascending = False)
AggFlights.show()

+------+----+-------------+--------------+
|Origin|Dest|Average_delay|Number_flights|
+------+----+-------------+--------------+
|   RDM| MFR|       1347.0|             2|
|   MDT| HPN|        798.0|             1|
|   ORD| GTF|        212.0|             1|
|   ICT| DAY|        210.0|             1|
|   ELM| ATL|        169.0|             2|
|   DSM| PIA|        168.0|             1|
|   ERI| ITH|        160.0|             1|
|   YNG| PIE|        141.0|             1|
|   CMH| HOU|        120.0|             1|
|   HRL| DAL|        111.0|             1|
|   PPG| HNL|        109.9|            35|
|   HNL| PPG|        105.9|            35|
|   PIE| YNG|        104.0|             1|
|   AVP| SFB|         93.0|             1|
|   ACY| MSY|         87.5|            11|
|   CPR| LAS|         85.0|             1|
|   LAS| CPR|         82.0|             1|
|   TTN| BNA|         76.5|            10|
|   MSP| PVD|         74.0|             1|
|   TUL| OKC|         69.0|             1|
+------+---

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [9]:
cleanFlightsDF.groupBy("ArrDelayCat").agg(
    F.count("*").alias("numVuelosCat")
).show()

+-----------+------------+
|ArrDelayCat|numVuelosCat|
+-----------+------------+
|     Slight|      298234|
|       None|     2004727|
|       Huge|      200152|
+-----------+------------+



In [10]:
cleanFlightsDF.groupBy("ArrDelayCat").count().show()

+-----------+-------+
|ArrDelayCat|  count|
+-----------+-------+
|     Slight| 298234|
|       None|2004727|
|       Huge| 200152|
+-----------+-------+



In [11]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin","Dest","ArrDelayCat").agg(
                                            F.count("*").alias("cuantos"))\
                                            .where("Origin = 'MCI' and Dest = 'MKE'")
sortedAvgDelaysDF.show()

+------+----+-----------+-------+
|Origin|Dest|ArrDelayCat|cuantos|
+------+----+-----------+-------+
|   MCI| MKE|       Huge|      8|
|   MCI| MKE|       None|    172|
|   MCI| MKE|     Slight|     18|
+------+----+-----------+-------+



## Vamos a desplegar ArrDelayCat

In [12]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin","Dest","ArrDelayCat").pivot("ArrDelayCat").agg(
                                            F.count("*").alias("cuantos"))
sortedAvgDelaysDF.show()

+------+----+-----------+----+----+------+
|Origin|Dest|ArrDelayCat|Huge|None|Slight|
+------+----+-----------+----+----+------+
|   BOS| LGB|       Huge|  22|null|  null|
|   BOS| HOU|       None|null| 194|  null|
|   BUF| JFK|     Slight|null|null|   108|
|   FLL| ORH|       None|null|  71|  null|
|   BOS| DEN|       Huge|  71|null|  null|
|   MSO| LAS|       None|null|  29|  null|
|   BIS| SFB|       None|null|  28|  null|
|   MSY| PIE|       None|null|  32|  null|
|   BOI| LAS|       None|null| 235|  null|
|   EWR| CVG|       None|null| 641|  null|
|   LEX| SFB|     Slight|null|null|    15|
|   ROA| CLT|     Slight|null|null|   100|
|   CLT| TLH|       None|null| 365|  null|
|   CLT| GRR|       None|null| 187|  null|
|   SFO| RDM|       None|null| 369|  null|
|   DEN| DFW|       None|null|1578|  null|
|   COS| ORD|       Huge|  52|null|  null|
|   LAN| MSP|       None|null| 203|  null|
|   MSP| RAP|     Slight|null|null|    44|
|   SBN| DTW|     Slight|null|null|    72|
+------+---

In [13]:
pivot = cleanFlightsDF.groupBy("Origin")\
                      .pivot("ArrDelayCat").agg(
                       F.count("*").alias("Conteo"),
                        F.max("ArrDelay").alias("maxArrDelay"))\
                        .sort("Origin")

pivot.show()

+------+-----------+----------------+-----------+----------------+-------------+------------------+
|Origin|Huge_Conteo|Huge_maxArrDelay|None_Conteo|None_maxArrDelay|Slight_Conteo|Slight_maxArrDelay|
+------+-----------+----------------+-----------+----------------+-------------+------------------+
|   ABE|        193|           674.0|       1230|            14.0|          239|              59.0|
|   ABI|         56|           397.0|        497|            14.0|           76|              59.0|
|   ABQ|        457|          1650.0|       7184|            14.0|          947|              59.0|
|   ABR|         22|           584.0|        198|            13.0|           20|              40.0|
|   ABY|         38|           641.0|        239|            14.0|           57|              50.0|
|   ACT|         40|           500.0|        386|            14.0|           42|              59.0|
|   ACV|         40|           900.0|        291|            14.0|           33|              59.0|


<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

In [14]:
AvgDelaysDF = cleanFlightsDF.groupBy("DayOfWeek").agg(F.avg("DepDelay").alias("RetrasoSalida"), F.avg("ArrDelay").alias("RetrasoLlegada")).orderBy("DayOfWeek")

AvgDelaysDF.show()

+---------+------------------+-------------------+
|DayOfWeek|     RetrasoSalida|     RetrasoLlegada|
+---------+------------------+-------------------+
|        1|10.430177708665964|  5.391113068725289|
|        2| 8.246502522185226| 2.8412409647873806|
|        3|  8.47071347600168| 3.0525338339576717|
|        4|  8.35856546210902| 2.7390527404801026|
|        5|10.220785437977693|  5.027363815430113|
|        6| 6.278199328016013|-0.5748593305876211|
|        7| 9.142161259888235| 3.2344449424598207|
+---------+------------------+-------------------+



In [15]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest").agg(
    F.mean("ArrDelay").alias("avgArrDelay")
)

In [16]:
AvgDelayDepAirport = cleanFlightsDF.groupBy("Origin").pivot("DayOfWeek").agg(F.avg("DepDelay").alias("RetrasoSalida"), F.avg("ArrDelay").alias("RetrasoLlegada"))

AvgDelayDepAirport.toPandas()

,Origin,1_RetrasoSalida,1_RetrasoLlegada,2_RetrasoSalida,2_RetrasoLlegada,3_RetrasoSalida,3_RetrasoLlegada,4_RetrasoSalida,4_RetrasoLlegada,5_RetrasoSalida,5_RetrasoLlegada,6_RetrasoSalida,6_RetrasoLlegada,7_RetrasoSalida,7_RetrasoLlegada
0,BGM,12.428571,5.500000,11.243902,5.951220,27.219512,23.097561,61.682927,60.097561,16.921053,13.868421,13.200000,11.240000,2.139535,-2.813953
1,PSE,5.918919,-4.416667,14.787879,10.545455,-0.685714,-9.800000,2.257143,0.685714,8.764706,-1.764706,9.647059,2.088235,7.606061,-4.151515
2,INL,-0.969697,-4.606061,14.944444,12.888889,0.647059,-1.235294,22.970588,20.088235,-9.558824,-11.588235,-6.935484,-13.129032,3.870968,3.967742
3,MSY,8.419930,3.062675,7.618881,2.519657,11.118672,5.621753,8.338354,2.813029,9.974827,5.146306,8.053008,2.720067,10.528926,5.273677
4,PPG,35.705882,35.058824,NaN,NaN,39.000000,31.000000,92.500000,99.166667,243.000000,238.454545,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,SAT,7.103980,0.380597,8.701169,2.891183,8.238300,2.128302,7.915839,0.968537,6.765349,1.716075,2.896138,-2.889481,7.411359,0.618085
352,STT,9.849462,-0.152174,-2.294118,-10.928571,3.957895,-7.106383,1.883721,-9.116279,14.155556,4.352273,2.611650,-9.820000,1.472527,-8.241758
353,RAP,22.342593,21.300926,7.837079,5.859551,4.900000,-0.468421,8.160194,4.058252,9.170213,7.882353,13.221519,7.279221,6.801020,3.261538
354,ASE,23.220000,18.057778,13.543529,8.528436,10.714922,3.703125,2.792325,-2.676471,14.357778,9.453333,17.795652,13.135667,15.370787,11.968397


In [17]:
AvgDelaysDaysDF = cleanFlightsDF.groupBy("Origin","Dest").pivot("DayOfWeek").agg(
                                            F.avg("DepDelay").alias("MediaRSal"),
                                            F.avg("ArrDelay").alias("MediaRLleg"))\
                                            .orderBy("1_MediaRSal", ascending = False)
AvgDelaysDaysDF.show()

+------+----+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+
|Origin|Dest|       1_MediaRSal|       1_MediaRLleg|        2_MediaRSal|       2_MediaRLleg|         3_MediaRSal|       3_MediaRLleg|        4_MediaRSal|       4_MediaRLleg|        5_MediaRSal|      5_MediaRLleg|        6_MediaRSal|      6_MediaRLleg|         7_MediaRSal|       7_MediaRLleg|
+------+----+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+
|   DSM| CID|             295.0|               null|               null|               null|                null|        

### Cuando utilizo pivot con una sola función de agregación, el alias es ignorado

In [18]:
flightsPd = flightsDF.groupby("Origin").pivot("DayOfWeek").agg(
    F.mean("ArrDelay").alias("MeanArrDelay")
).sort("Origin").toPandas()

### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

In [19]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin","Dest").pivot("DayOfWeek").agg(
                                            F.avg("DepDelay").alias("MediaRSal"),
                                            F.avg("ArrDelay").alias("MediaRLleg"))\
                                            .orderBy("1_MediaRSal", ascending = False)
averageDelayOriginDestDF.show()

+------+----+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+
|Origin|Dest|       1_MediaRSal|       1_MediaRLleg|        2_MediaRSal|       2_MediaRLleg|         3_MediaRSal|       3_MediaRLleg|        4_MediaRSal|       4_MediaRLleg|        5_MediaRSal|      5_MediaRLleg|        6_MediaRSal|      6_MediaRLleg|         7_MediaRSal|       7_MediaRLleg|
+------+----+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+
|   DSM| CID|             295.0|               null|               null|               null|                null|        

In [20]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest")\
                                         .agg(F.mean("ArrDelay").alias("avgArrDelay"))
averageDelayOriginDestDF.show(10)

+------+----+------------------+
|Origin|Dest|       avgArrDelay|
+------+----+------------------+
|   BQN| MCO| 5.762711864406779|
|   PBI| DCA|-0.391025641025641|
|   SPI| ORD|15.590909090909092|
|   STS| PHX|12.724409448818898|
|   MDW| MEM|3.3542600896860986|
|   PHL| MCO| 7.520783847980997|
|   SMF| BUR| 7.034831460674157|
|   SNA| PHX| 6.482380579483164|
|   ATL| GSP|-1.734910277324633|
|   LAS| LIT| 5.218487394957983|
+------+----+------------------+
only showing top 10 rows



In [21]:
cleanFlightsDF.show(10)

+-----+----------+---------+----------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|Month|DayofMonth|DayOfWeek|FlightDate|Origin|  OriginCity|Dest|    DestCity|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|CancellationCode|Diverted|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCat|
+-----+----------+---------+----------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|    1|        14|        7|2018-01-14|   SYR|Syracuse, NY| DTW| Detroit, MI|   null|    null|   null|    null|      1.0|               B|     0.0|             null|   null|   374.0|        null|        null|    nu

In [22]:
joinedDF = cleanFlightsDF.join(averageDelayOriginDestDF,
                              on = ["Origin", "Dest"])\
                        .withColumn("belowAverage", F.col("ArrDelay") < F.col("avgArrDelay"))
joinedDF.show(10)

+------+----+-----+----------+---------+----------+-----------+---------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+------------------+------------+
|Origin|Dest|Month|DayofMonth|DayOfWeek|FlightDate| OriginCity| DestCity|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|CancellationCode|Diverted|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCat|       avgArrDelay|belowAverage|
+------+----+-----+----------+---------+----------+-----------+---------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+------------------+------------+
|   ATL| GSP|    1|         3|        3|2018-01-03|Atlanta, GA|Greer, SC|   1024|    -1.0|   1114|    -6.0|      0.0|            n

In [23]:
tempDF = cleanFlightsDF.alias("a").join(averageDelayOriginDestDF.alias("b"),
                            on = (cleanFlightsDF["Origin"] == averageDelayOriginDestDF["Origin"]) & 
                                  (cleanFlightsDF["Dest"] == averageDelayOriginDestDF["Dest"])
                                , how = "left_outer")\
                        .select(F.col("a.Origin"), F.col("b.Dest"), "DayOfWeek", "ArrDelay", "avgArrDelay")

tempDF.show()
tempDF.printSchema()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|       avgArrDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|
|   ATL| GSP|        3|    47.0|-1.734910277324633|
|   ATL| GSP|        4|    null|-1.734910277324633|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|
|   ATL| GSP|        4|    38.0|-1.734910277324633|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        1|    14.0|-1.734910277324633|
|   ATL| GSP|        1|    -3.0|-1.734910277324633|
|   ATL| GSP|        2|   -22.0|-1.734910277324633|
|   ATL| GSP

In [24]:
averageDelayOriginDestDFrenombrado = averageDelayOriginDestDF.withColumnRenamed("Origin", "OriginDerecha")\
                    .withColumnRenamed("Dest", "DestDerecha")

tempDF = cleanFlightsDF.join(averageDelayOriginDestDFrenombrado,
                            on = (cleanFlightsDF["Origin"] == averageDelayOriginDestDFrenombrado["OriginDerecha"]) & 
                                  (cleanFlightsDF["Dest"] == averageDelayOriginDestDFrenombrado["DestDerecha"]))\
                        .select(cleanFlightsDF["Origin"], cleanFlightsDF["Dest"], "DayOfWeek", "ArrDelay", "avgArrDelay")

tempDF.show()
tempDF.printSchema()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|       avgArrDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|
|   ATL| GSP|        3|    47.0|-1.734910277324633|
|   ATL| GSP|        4|    null|-1.734910277324633|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|
|   ATL| GSP|        4|    38.0|-1.734910277324633|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        1|    14.0|-1.734910277324633|
|   ATL| GSP|        1|    -3.0|-1.734910277324633|
|   ATL| GSP|        2|   -22.0|-1.734910277324633|
|   ATL| GSP

In [25]:
from pyspark.sql import Window
w = Window().partitionBy("Origin", "Dest")
windowDF = cleanFlightsDF.withColumn("avgArrDelay", F.mean("ArrDelay").over(w))\
                         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgArrDelay")\
                         .withColumn("belowAverage", F.col("ArrDelay")<F.col("avgArrDelay"))
windowDF.show()

+------+----+---------+--------+------------------+------------+
|Origin|Dest|DayOfWeek|ArrDelay|       avgArrDelay|belowAverage|
+------+----+---------+--------+------------------+------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    47.0|-1.734910277324633|       false|
|   ATL| GSP|        4|    null|-1.734910277324633|        null|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        4|    38.0|-1.734910277324633|       false|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|        true|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|        true|
|   ATL| GSP|        1|    10.0|-1.734910277324633|       false|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|        true|
|   ATL| GSP|        1|  

In [26]:
flightsDF.groupby("Origin").pivot("DayOfWeek").agg(
    F.mean("ArrDelay").alias("MeanArrDelay"),
    F.min("ArrDelay").alias("MinArrDelay")
    ).sort("Origin").toPandas()

,Origin,1_MeanArrDelay,1_MinArrDelay,2_MeanArrDelay,2_MinArrDelay,3_MeanArrDelay,3_MinArrDelay,4_MeanArrDelay,4_MinArrDelay,5_MeanArrDelay,5_MinArrDelay,6_MeanArrDelay,6_MinArrDelay,7_MeanArrDelay,7_MinArrDelay
0,ABE,14.953307,-41.0,13.893519,-32.0,15.276786,-34.0,5.924370,-42.0,18.183333,-37.0,6.940299,-33.0,10.027149,-36.0
1,ABI,10.650000,-28.0,16.364706,-27.0,-0.547619,-23.0,0.376344,-22.0,4.641304,-27.0,16.492063,-23.0,4.606383,-28.0
2,ABQ,1.442438,-67.0,0.363406,-61.0,1.736409,-67.0,3.111111,-61.0,-0.176892,-52.0,-1.655721,-63.0,-0.252019,-59.0
3,ABR,10.571429,-27.0,12.468750,-33.0,6.294118,-31.0,16.441176,-28.0,8.529412,-29.0,0.125000,-31.0,9.117647,-30.0
4,ABY,14.207547,-34.0,3.978723,-34.0,12.804348,-20.0,10.645833,-25.0,9.040816,-21.0,29.562500,-17.0,27.234043,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,XNA,9.592857,-39.0,9.375580,-46.0,6.838462,-49.0,1.854015,-43.0,3.877941,-50.0,-0.977208,-51.0,5.438861,-41.0
352,YAK,-9.451613,-45.0,-13.000000,-41.0,-10.727273,-46.0,2.848485,-34.0,-9.470588,-36.0,-12.088235,-50.0,-10.090909,-41.0
353,YKM,0.338462,-16.0,-0.490909,-21.0,5.666667,-28.0,1.359375,-18.0,6.269841,-18.0,5.941176,-21.0,1.603175,-30.0
354,YNG,9.000000,9.0,NaN,NaN,NaN,NaN,141.000000,141.0,NaN,NaN,NaN,NaN,NaN,NaN


**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [27]:
from pyspark.sql import Window
w = Window().partitionBy("Origin", "Dest")
windowDF = cleanFlightsDF.withColumn("avgArrDelay", F.mean("ArrDelay").over(w))\
                         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgArrDelay")\
                         .withColumn("belowAverage", F.col("ArrDelay")<F.col("avgArrDelay"))
windowDF.show()

+------+----+---------+--------+------------------+------------+
|Origin|Dest|DayOfWeek|ArrDelay|       avgArrDelay|belowAverage|
+------+----+---------+--------+------------------+------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        3|    47.0|-1.734910277324633|       false|
|   ATL| GSP|        4|    null|-1.734910277324633|        null|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        4|    38.0|-1.734910277324633|       false|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|        true|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|        true|
|   ATL| GSP|        1|    10.0|-1.734910277324633|       false|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|        true|
|   ATL| GSP|        1|  

### Inciso: como crear un DF de Spark a partir de un dataframe

In [28]:
import pandas as pd
airports_pd = pd.DataFrame({"IATA":["JFK", "LIT", "SEA"], 
                            "Year": [1948, 1931, 1949]})

airportsFromPandasDF = spark.createDataFrame(airports_pd)
airportsFromPandasDF.printSchema()
airportsFromPandasDF.show()

root
 |-- IATA: string (nullable = true)
 |-- Year: long (nullable = true)

+----+----+
|IATA|Year|
+----+----+
| JFK|1948|
| LIT|1931|
| SEA|1949|
+----+----+



In [29]:
spark.createDataFrame(["10","11","13"], "string").toDF("age").show()

+---+
|age|
+---+
| 10|
| 11|
| 13|
+---+



In [30]:
data = [['JFK', 1948], ['LIT', 1931], ['Ann', 1949]]
pandasDF = pd.DataFrame(data, columns = ['IATA','Year'])
airportsFromPandasDF2 = spark.createDataFrame(data)
airportsFromPandasDF2.printSchema()
airportsFromPandasDF2.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+----+
| _1|  _2|
+---+----+
|JFK|1948|
|LIT|1931|
|Ann|1949|
+---+----+



In [31]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])


In [32]:
joinedFlightsDF = cleanFlightsDF.join(airportsDF,
                                on = cleanFlightsDF.Origin == airportsDF.IATA,
                                how = "left_outer")

# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.filter(~(F.col("FullName").isNull()))\
                        .select("Origin", "Dest", "IATA", "FullName", "Year")\
                        .show(10,truncate = False)

+------+----+----+------------------------------------+----+
|Origin|Dest|IATA|FullName                            |Year|
+------+----+----+------------------------------------+----+
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
+------+----+----+------------------------------------+----+
only showing top 10 rows



## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [33]:
from pyspark.sql.functions import udf
from pyspark.sql import types as T

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, T.DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0
+------+----+--------+------------------+
|Origin|Dest|Distance|            DistKM|
+------+----+--------+------------------+
|   CLE| JFK|   425.0|             680.0|
|   MCO| PSE|  1179.0|            1886.4|
|   FLL| DTW|  1127.0|            1803.2|
|   LAX| MTJ|   666.0|1065.6000000000001|
|   AZA| MLI|  1288.0|            2060.8|
+------+----+--------+------------------+
only showing top 5 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [37]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daysOfWeek[dayInteger]
    
# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = F.udf(dayOfWeekToString, StringType())



In [38]:
# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()


+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   BQK| ATL|        4|       Thursday|
|   CVG| PHL|        3|      Wednesday|
|   DTW| DFW|        5|         Friday|
|   SEA| JFK|        2|        Tuesday|
|   JAX| JFK|        2|        Tuesday|
|   RDU| BOS|        3|      Wednesday|
|   SEA| BOS|        3|      Wednesday|
|   AUS| FLL|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   SLC| BOS|        6|       Saturday|
|   BOS| HOU|        6|       Saturday|
|   BDL| MCO|        7|         Sunday|
|   SJU| TPA|        7|         Sunday|
|   PGD| TYS|        6|       Saturday|
|   PIE| CVG|        6|       Saturday|
|   ABE| SFB|        7|         Sunday|
|   LAS| BIS|        7|         Sunday|
|   ROC| PGD|        1|         Monday|
|   EWR| CVG|        1|         Monday|
|   CVG| SFB|        1|         Monday|
+------+----+---------+---------------+
only showing top 20 rows

